In [1]:
import pandas as pd
import numpy as np

In [2]:
conditions = pd.read_csv('../Case Files/sample_date_csv/conditions.csv')
print('Total shape : ',conditions.shape)
conditions.head()

Total shape :  (8376, 6)


,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION
0,2001-05-01,NaN,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder)
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,444814009,Viral sinusitis (disorder)
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,195662009,Acute viral pharyngitis (disorder)
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,10509002,Acute bronchitis (disorder)
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,195662009,Acute viral pharyngitis (disorder)


In [3]:
# Start : The date the condition was diagnosed.
# Stop : The date the condition resolved, if applicable.

In [4]:
# calculating number of NULL in STOP
conditions['STOP'].isnull().sum()
# total 3811 are such records in which stop is NAN
# From these records, we can say that here the condition was not resolved, hence it is non - adherence

3811

In [5]:
total_patients_conditions = conditions['PATIENT'].nunique()
total_patients_non_adherant = conditions[conditions['STOP'].isnull()]['PATIENT'].nunique()
print('Total unique patients in conditions.csv : ',total_patients_conditions)
print('Total unique patients whose condition was not resolved (non - adherent patients) : ',total_patients_non_adherant)
print('Total unique patients whose condition was resolved (adherent patients) : ',total_patients_conditions-total_patients_non_adherant)

Total unique patients in conditions.csv :  1152
Total unique patients whose condition was not resolved (non - adherent patients) :  922
Total unique patients whose condition was resolved (adherent patients) :  230


In [6]:
# 230 patients are adherent
# 922 patients are non - adherent
# creating a new column , isadherent : which will tell is that particular patient is adherent or not

In [7]:
conditions['isadherent'] = conditions['STOP'].notnull()*1
conditions.head()

,START,STOP,PATIENT,ENCOUNTER,CODE,DESCRIPTION,isadherent
0,2001-05-01,NaN,1d604da9-9a81-4ba9-80c2-de3375d59b40,8f104aa7-4ca9-4473-885a-bba2437df588,40055000,Chronic sinusitis (disorder),0
1,2011-08-09,2011-08-16,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,9d35ec9f-352a-4629-92ef-38eae38437e7,444814009,Viral sinusitis (disorder),1
2,2011-11-16,2011-11-26,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd,195662009,Acute viral pharyngitis (disorder),1
3,2011-05-13,2011-05-27,10339b10-3cd1-4ac3-ac13-ec26728cb592,e1ab4933-07a1-49f0-b4bd-05500919061d,10509002,Acute bronchitis (disorder),1
4,2011-02-06,2011-02-14,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,b8f76eba-7795-4dcd-a544-f27ac2ef3d46,195662009,Acute viral pharyngitis (disorder),1


In [8]:
# Extracting patients and isadherent columns seperately
patients_info = conditions[['PATIENT', 'isadherent']]
patients_info.head()

,PATIENT,isadherent
0,1d604da9-9a81-4ba9-80c2-de3375d59b40,0
1,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1
2,8d4c4326-e9de-4f45-9a4c-f8c36bff89ae,1
3,10339b10-3cd1-4ac3-ac13-ec26728cb592,1
4,f5dcd418-09fe-4a2f-baa0-3da800bd8c3a,1


In [9]:
#removing duplicates
patients_info = patients_info.drop_duplicates()
patients_info.shape

(2041, 2)

In [10]:
print('But total unique patients in conditions.csv : ',total_patients_conditions)

But total unique patients in conditions.csv :  1152


In [11]:
# this means that 889(2041 - 1152) are such records in which the patient is sometimes adherent and sometimes not isadherent
# if a patient is not adherent even once, he should fall in the category of non-adherent

In [12]:
group = patients_info.groupby('PATIENT')
patients_info = group.apply(lambda x : x['isadherent'].unique())
patients_info = patients_info.reset_index() 
patients_info.rename(columns = {0:'isadherent'}, inplace = True)

In [13]:
patients_info.head()

,PATIENT,isadherent
0,00185faa-2760-4218-9bf5-db301acf8274,"[1, 0]"
1,0042862c-9889-4a2e-b782-fac1e540ecb4,[1]
2,0047123f-12e7-486c-82df-53b3a450e365,"[1, 0]"
3,010d4a3a-2316-45ed-ae15-16f01c611674,"[1, 0]"
4,0149d553-f571-4e99-867e-fcb9625d07c2,"[0, 1]"


In [14]:
# columns having value of isadherent are those patients which are sometimes adherent and sometimes not
# Hence first calculating the frequency of column and then appending that column

In [15]:
patients_info1 = group.apply(lambda x : x['isadherent'].nunique())
patients_info1 = patients_info1.reset_index() 
patients_info1.rename(columns = {0:'freq_isadherent'}, inplace = True)
freq_isadherent = patients_info1['freq_isadherent']
patients_info = pd.concat([patients_info, freq_isadherent], axis=1)
patients_info.head()

,PATIENT,isadherent,freq_isadherent
0,00185faa-2760-4218-9bf5-db301acf8274,"[1, 0]",2
1,0042862c-9889-4a2e-b782-fac1e540ecb4,[1],1
2,0047123f-12e7-486c-82df-53b3a450e365,"[1, 0]",2
3,010d4a3a-2316-45ed-ae15-16f01c611674,"[1, 0]",2
4,0149d553-f571-4e99-867e-fcb9625d07c2,"[0, 1]",2


In [16]:
# The columns having freq_isadherent mare than 1 are those non adherent patients

In [17]:
patients_info.loc[patients_info['freq_isadherent'] > 1, 'isadherent'] = 0
patients_info.drop('freq_isadherent',axis = 1,inplace = True)
print(patients_info['isadherent'].value_counts())

0      889
[1]    230
[0]     33
Name: isadherent, dtype: int64


In [18]:
# converting [0] -> 0 and [1] -> 1
patients_info.loc[patients_info['isadherent'] == 0, 'isadherent'] = 0
patients_info.loc[patients_info['isadherent'] == 1, 'isadherent'] = 1
patients_info['isadherent'].value_counts()

0    922
1    230
Name: isadherent, dtype: int64

In [19]:
patients_info.to_csv('../myCSV/isadherent.csv',index=False)